In [2]:
# Load packages
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

from models import NeuralNetwork, RNNClassifier, TrainConfig, evaluate, save_model, load_model, plot_results
from utils import load_data, split_data, encode_data, check_cuda_memory, PositionalEncoder
from pathlib import Path
import altair as alt
import pandas as pd

from tqdm import tqdm

if torch.cuda.is_available():
    print("Device: cuda")
    check_cuda_memory()
else:
    print("Device: cpu")

models_dir = Path('models/gb')


Device: cuda
NVIDIA GeForce RTX 3050 Laptop GPU
Total: 4,091.48 MB
Reserved: 0.00 MB
Allocated: 0.00 MB
Free in reserved: 0.00 MB


# Load data

In [3]:
data = load_data(folder_path="data/train/power/", file_list=['power-hr-train.tsv'],text_head='text')
train_raw, test_raw = split_data(data, test_size=0.2, random_state=0)

print(f"Data size: {len(data)}, % positive class: {sum(data.labels) / len(data) * 100:.2f}%")


Data size: 10741, % positive class: 39.73%


# Encode

In [4]:

print("Prepare words_encoder...")
words_encoder = TfidfVectorizer(max_features=50000)
words_encoder.fit(train_raw.texts)

print("Prepare chars_encoder...")
chars_encoder = TfidfVectorizer(max_features=50000, analyzer="char", ngram_range=(3,5), use_idf=True, sublinear_tf=True)
chars_encoder.fit(train_raw.texts)

Prepare words_encoder...
Prepare chars_encoder...


TfidfVectorizer(analyzer='char', max_features=50000, ngram_range=(3, 5),
                sublinear_tf=True)

# Neural Networks

### Word feature

In [4]:
print("Prepare data...")
train_nn_words = encode_data(train_raw, words_encoder)
test_nn_words = encode_data(test_raw, words_encoder)

print("Train model")

if not models_dir.exists():
    models_dir.mkdir(parents=True, exist_ok=True)

train_config = TrainConfig(
    num_epochs      = 10,
    early_stop      = False,
    violation_limit = 5,
    
)

dataloader = DataLoader(train_nn_words, batch_size=128, shuffle=True)

USE_CACHE = False

model_nn_words = NeuralNetwork(
    input_size=len(words_encoder.vocabulary_),
    hidden_size=128,
    device='cuda'
)

if (models_dir / 'model_nn_words.pt').exists() and USE_CACHE:
    model_nn_words = load_model(model_nn_words, models_dir, 'model_nn_words')
else:
    model_nn_words.fit(dataloader, train_config, disable_progress_bar=False)
    save_model(model_nn_words, models_dir, "model_nn_words")


# Evaluate
with torch.no_grad():
    X_test_nn = torch.stack([test[0] for test in test_nn_words]).cpu()
    y_test_nn = torch.stack([test[1] for test in test_nn_words]).cpu()
    y_pred_nn_words = model_nn_words.predict(X_test_nn)
    logits_nn_words = model_nn_words.forward(X_test_nn)

result_nn_words = evaluate(y_test_nn.cpu(), y_pred_nn_words.cpu(), logits_nn_words.cpu())

# Plot training accuracy and loss side-by-side
plot_results(model_nn_words, train_config, dataloader)

model_nn_words.to('cpu')
torch.cuda.empty_cache()

Prepare data...
Train model



Epoch 10: 100%|██████████| 68/68 [00:00<00:00, 72.62batch/s, batch_accuracy=1, loss=5.25e-5] 


Accuracy: 0.6667, Precision: 0.5624, Recall: 0.5411, F1: 0.5515, AUC: 0.6914


alt.HConcatChart(...)

### Char features

In [5]:
print("Prepare data...")
test_nn_chars = encode_data(test_raw, chars_encoder)

print("Train model")
models_dir = Path('models/gb')

if not models_dir.exists():
    models_dir.mkdir(parents=True, exist_ok=True)

train_config = TrainConfig(
    num_epochs      = 10,
    early_stop      = False,
    violation_limit = 5,
    
)



USE_CACHE = False

model_nn_chars = NeuralNetwork(
    input_size=len(chars_encoder.vocabulary_),
    hidden_size=128,
    device='cuda'
)

if (models_dir / 'model_nn_chars.pt').exists() and USE_CACHE:
    model_nn_chars = load_model(model_nn_chars, models_dir, 'model_nn_chars')
else:
    train_nn_chars = encode_data(train_raw, chars_encoder)
    dataloader = DataLoader(train_nn_chars, batch_size=128, shuffle=True)

    model_nn_chars.fit(dataloader, train_config, disable_progress_bar=False)
    save_model(model_nn_chars, models_dir, "model_nn_chars")


with torch.no_grad():
    X_test = torch.stack([test[0] for test in test_nn_chars]).to(model_nn_chars.device)
    y_test = torch.stack([test[1] for test in test_nn_chars]).to(model_nn_chars.device)
    y_pred = model_nn_chars.predict(X_test)
    logits = model_nn_chars.forward(X_test)

result_nn_chars = evaluate(y_test.cpu(), y_pred.cpu(), logits.cpu())

# Plot training accuracy and loss side-by-side
plot_results(model_nn_chars, train_config, dataloader)

model_nn_words.to('cpu')
torch.cuda.empty_cache()

Prepare data...
Train model



Epoch 10: 100%|██████████| 68/68 [00:01<00:00, 67.39batch/s, batch_accuracy=1, loss=0.000151]


Accuracy: 0.6834, Precision: 0.5840, Recall: 0.5714, F1: 0.5776, AUC: 0.7158


alt.HConcatChart(...)

# RNN

### Word feature

In [10]:

print("Prepare data encoder...")
rnn_words_encoder = PositionalEncoder()
rnn_words_encoder.fit(train_raw.texts)

train_dataloader = DataLoader(train_raw, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_raw, batch_size=128, shuffle=False)

test_inputs = rnn_words_encoder.transform(test_raw.texts)

# Prepare baseline config
train_config = TrainConfig(
    optimizer_params = {'lr': 0.01},
    num_epochs       = 10,
    early_stop       = False,
    violation_limit  = 5
)

# Train baseline model
model_lstm_words = RNNClassifier(
    rnn_network         = nn.LSTM,
    word_embedding_dim  = 32,
    hidden_dim          = 64,
    bidirectional       = False,
    dropout             = 0,
    encoder             = rnn_words_encoder,
    device              = 'cuda'
)

USE_CACHE = False


if (models_dir / 'model_lstm_words.pt').exists() and USE_CACHE:
    model_lstm_words = load_model(model_lstm_words, 'model_lstm_words')
else:
    model_lstm_words.fit(train_dataloader, train_config, no_progress_bar=False)
    save_model(model_lstm_words, models_dir, "model_lstm_words")

# Evaluate
with torch.no_grad():
    model_lstm_words.device = "cpu"
    model_lstm_words.cpu()

    pred_LSTM_words = []
    logits_LSTM_words = []

    for _, _, raw_inputs, raw_targets in test_dataloader:
        batch_encoder = PositionalEncoder(vocabulary=rnn_words_encoder.vocabulary)
        test_inputs = batch_encoder.fit_transform(raw_inputs).cpu()
        test_targets = torch.as_tensor(raw_targets, dtype=torch.float).cpu()  # nn.CrossEntropyLoss() require target to be float

        pred_LSTM_words.append(model_lstm_words.predict(test_inputs))
        logits_LSTM_words.append(model_lstm_words.forward(test_inputs))

pred_LSTM_words = torch.concat(pred_LSTM_words).numpy()
logits_LSTM_words = torch.concat(logits_LSTM_words).numpy()


model_lstm_words_result = evaluate(test_raw.labels, pred_LSTM_words, logits_LSTM_words)
# print(model_lstm_words_result)

np.save(models_dir / 'model_lstm_words_results.npy', model_lstm_words_result)



Prepare data encoder...


Epoch 10: 100%|██████████| 68/68 [00:32<00:00,  2.12batch/s, batch_accuracy=1, loss=111]    


Accuracy: 0.6250, Precision: 0.5072, Recall: 0.3565, F1: 0.4187, AUC: 0.6305


In [11]:

print("Prepare data encoder...")
rnn_chars_encoder = PositionalEncoder(tokenizer=chars_encoder.build_analyzer())
rnn_chars_encoder.fit(train_raw.texts)

train_dataloader = DataLoader(train_raw, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_raw, batch_size=128, shuffle=False)

test_inputs = rnn_chars_encoder.transform(test_raw.texts)

# Prepare baseline config
train_config = TrainConfig(
    optimizer_params = {'lr': 0.01},
    num_epochs       = 10,
    early_stop       = False,
    violation_limit  = 5
)

# Train baseline model
model_lstm_chars = RNNClassifier(
    rnn_network         = nn.LSTM,
    word_embedding_dim  = 32,
    hidden_dim          = 64,
    bidirectional       = False,
    dropout             = 0,
    encoder             = rnn_chars_encoder,
    device              = 'cuda'
)

USE_CACHE = False

if (models_dir / 'model_lstm_chars.pt').exists() and USE_CACHE:
    model_lstm_chars = load_model(model_lstm_chars, 'model_lstm_chars')
else:
    model_lstm_chars.fit(train_dataloader, train_config, no_progress_bar=False)
    save_model(model_lstm_chars, models_dir, "model_lstm_chars")

with torch.no_grad():
    model_lstm_chars.device = "cpu"
    model_lstm_chars.cpu()

    pred_LSTM_chars = []
    logits_LSTM_chars = []

    for _, _, raw_inputs, raw_targets in test_dataloader:
        batch_encoder = PositionalEncoder(vocabulary=rnn_chars_encoder.vocabulary)
        test_inputs = batch_encoder.fit_transform(raw_inputs).cpu()
        test_targets = torch.as_tensor(raw_targets, dtype=torch.float).cpu()  # nn.CrossEntropyLoss() require target to be float


        pred_LSTM_chars.append(model_lstm_chars.predict(test_inputs))
        logits_LSTM_chars.append(model_lstm_chars.forward(test_inputs))

pred_LSTM_chars = torch.concat(pred_LSTM_chars).numpy()
logits_LSTM_chars = torch.concat(logits_LSTM_chars).numpy()

model_lstm_chars_result = evaluate(test_raw.labels, pred_LSTM_chars, logits_LSTM_chars)
# print(model_lstm_chars_result)

np.save(models_dir / 'model_lstm_chars_results.npy', model_lstm_chars_result)



Prepare data encoder...


Epoch 10: 100%|██████████| 68/68 [00:31<00:00,  2.14batch/s, batch_accuracy=0.558, loss=104]


Accuracy: 0.4411, Precision: 0.3923, Recall: 0.8660, F1: 0.5400, AUC: 0.5439


# Other classifiers from sklearn

https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

"Particularly in high-dimensional spaces, data can more easily be separated linearly and the simplicity of classifiers such as naive Bayes and linear SVMs might lead to better generalization than is achieved by other classifiers."

In [6]:
# Prepare train & test set
X_train_skl_words = words_encoder.transform(train_raw.texts)
X_test_skl_words = words_encoder.transform(test_raw.texts)

X_train_skl_chars = chars_encoder.transform(train_raw.texts)
X_test_skl_chars = chars_encoder.transform(test_raw.texts)

## Linear SVC
Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

LinearSVC with TfIdf did good on balanced English

### Word feature

In [7]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# LinearSVC, word


print("Fit model")
base_svc = LinearSVC()
model_LinearSVC_words = CalibratedClassifierCV(estimator=base_svc, cv=5)
model_LinearSVC_words.fit(X_train_skl_words, train_raw.labels)

pred_LinearSVC_words = model_LinearSVC_words.predict(X_test_skl_words)
logits_linearSVC_words = model_LinearSVC_words.predict_proba(X_test_skl_words)

result_linearSVC_words =  evaluate(test_raw.labels, pred_LinearSVC_words, logits_linearSVC_words[:, 1])

Fit model
Accuracy: 0.6930, Precision: 0.6190, Recall: 0.4930, F1: 0.5489, AUC: 0.7136


In [8]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

print("Fit model")
base_svc = LinearSVC()
model_LinearSVC_chars = CalibratedClassifierCV(estimator=base_svc, cv=5)
model_LinearSVC_chars.fit(X_train_skl_chars, train_raw.labels)

pred_LinearSVC_chars = model_LinearSVC_chars.predict(X_test_skl_chars)
logits_linearSVC_chars = model_LinearSVC_chars.predict_proba(X_test_skl_chars)

result_linearSVC_chars =  evaluate(test_raw.labels, pred_LinearSVC_chars, logits_linearSVC_chars[:, 1])

Fit model
Accuracy: 0.7136, Precision: 0.6524, Recall: 0.5221, F1: 0.5801, AUC: 0.7407


# Logistic Regression


In [9]:
# Word features

from sklearn.linear_model import LogisticRegression

print("Fit model")
model_logreg_words = LogisticRegression()
model_logreg_words.fit(X_train_skl_words, train_raw.labels)

pred_logreg_words = model_logreg_words.predict(X_test_skl_words)
logits_logreg_words = model_logreg_words.predict_proba(X_test_skl_words)

result_linearlogreg_words =  evaluate(test_raw.labels, pred_logreg_words, logits_logreg_words[:, 1])

Fit model
Accuracy: 0.7016, Precision: 0.6511, Recall: 0.4576, F1: 0.5375, AUC: 0.7307


In [10]:
# char features

from sklearn.linear_model import LogisticRegression

print("Fit model")
model_logreg_chars = LogisticRegression()
model_logreg_chars.fit(X_train_skl_chars, train_raw.labels)

pred_logreg_chars = model_logreg_chars.predict(X_test_skl_chars)
logits_logreg_chars = model_logreg_chars.predict_proba(X_test_skl_chars)

result_linearlogreg_chars =  evaluate(test_raw.labels, pred_logreg_chars, logits_logreg_chars[:, 1])

Fit model
Accuracy: 0.7126, Precision: 0.6616, Recall: 0.4943, F1: 0.5658, AUC: 0.7570


## SGDClassifier
SGD requires a number of hyperparameters such as the regularization parameter and the number of iterations.

SGD is sensitive to feature scaling.

In [11]:
# word features

from sklearn.linear_model import SGDClassifier

print("Fit model")
model_sgd_words = SGDClassifier(loss='log_loss')
model_sgd_words.fit(X_train_skl_words, train_raw.labels)

pred_sgd_words = model_sgd_words.predict(X_test_skl_words)
logits_sgd_words = model_sgd_words.predict_proba(X_test_skl_words)

result_linearsgd_words =  evaluate(test_raw.labels, pred_sgd_words, logits_sgd_words[:, 1])

Fit model
Accuracy: 0.7031, Precision: 0.6734, Recall: 0.4197, F1: 0.5171, AUC: 0.7286


In [12]:
# chars features

from sklearn.linear_model import SGDClassifier

print("Fit model")
model_sgd_chars = SGDClassifier(loss='log_loss')
model_sgd_chars.fit(X_train_skl_chars, train_raw.labels)

pred_sgd_chars = model_sgd_chars.predict(X_test_skl_chars)
logits_sgd_chars = model_sgd_chars.predict_proba(X_test_skl_chars)

result_linearsgd_chars =  evaluate(test_raw.labels, pred_sgd_chars, logits_sgd_chars[:, 1])

Fit model
Accuracy: 0.7150, Precision: 0.6576, Recall: 0.5171, F1: 0.5789, AUC: 0.7564


## Naive Bayes

Overall bad performance, not worth pursuing

In [13]:
# words features

from sklearn.naive_bayes import GaussianNB

print("Fit model")
model_gnb_words = GaussianNB()
model_gnb_words.fit(X_train_skl_words.toarray(), train_raw.labels)

pred_gnb_words = model_gnb_words.predict(X_test_skl_words.toarray())
logits_gnb_words = model_gnb_words.predict_proba(X_test_skl_words.toarray())

result_lineargnb_words =  evaluate(test_raw.labels, pred_gnb_words, logits_gnb_words[:, 1])

Fit model
Accuracy: 0.5972, Precision: 0.4681, Recall: 0.4640, F1: 0.4660, AUC: 0.5723


In [14]:
# chars features

from sklearn.naive_bayes import GaussianNB

print("Fit model")
model_gnb_chars = GaussianNB()
model_gnb_chars.fit(X_train_skl_chars.toarray(), train_raw.labels)

pred_gnb_chars = model_gnb_chars.predict(X_test_skl_chars.toarray())
logits_gnb_chars = model_gnb_chars.predict_proba(X_test_skl_chars.toarray())

result_lineargnb_chars =  evaluate(test_raw.labels, pred_gnb_chars, logits_gnb_chars[:, 1])

Fit model
Accuracy: 0.6810, Precision: 0.5767, Recall: 0.5942, F1: 0.5853, AUC: 0.6972


# Xgboost


In [15]:
import xgboost as xgb
import pandas as pd

# Prepare data for xgboost
train_dmat_words = xgb.DMatrix(X_train_skl_words, pd.array(train_raw.labels).astype('category'))
test_dmat_words = xgb.DMatrix(X_test_skl_words, pd.array(test_raw.labels).astype('category'))

train_dmat_chars = xgb.DMatrix(X_train_skl_chars, pd.array(train_raw.labels).astype('category'))
test_dmat_chars = xgb.DMatrix(X_test_skl_chars, pd.array(test_raw.labels).astype('category'))

### Word features

In [16]:

params = {
    "booster": "gbtree",
    "objective": "binary:logistic",  # there is also binary:hinge but hinge does not output probability
    "tree_method": "hist",  # default to hist
    "device": "cuda",

    # Params for tree booster
    "eta": 0.3,
    "gamma": 0.0,  # Min loss achieved to split the tree
    "max_depth": 6,
    "reg_alpha": 0,
    "reg_lambda": 1,

}
evals_words = [(train_dmat_words, "train")]
iterations = 2000

model_xgb_words = xgb.train(
    params = params,
    dtrain = train_dmat_words,
    num_boost_round = iterations,
    evals = evals_words,
    verbose_eval = 100
)

pred_xgb_words_probs = model_xgb_words.predict(test_dmat_words)
result_xgb_words = evaluate(test_raw.labels, pred_xgb_words_probs > 0.5, pred_xgb_words_probs)

[0]	train-logloss:0.62021
[100]	train-logloss:0.19892
[200]	train-logloss:0.10027
[300]	train-logloss:0.05127
[400]	train-logloss:0.02922
[500]	train-logloss:0.01756
[600]	train-logloss:0.01152
[700]	train-logloss:0.00825
[800]	train-logloss:0.00615
[900]	train-logloss:0.00485
[1000]	train-logloss:0.00380
[1100]	train-logloss:0.00309
[1200]	train-logloss:0.00252
[1300]	train-logloss:0.00213
[1400]	train-logloss:0.00183
[1500]	train-logloss:0.00160
[1600]	train-logloss:0.00143
[1700]	train-logloss:0.00129
[1800]	train-logloss:0.00119
[1900]	train-logloss:0.00110
[1999]	train-logloss:0.00104
Accuracy: 0.7031, Precision: 0.6290, Recall: 0.5272, F1: 0.5736, AUC: 0.7364


In [17]:
# Can use only half of the original max features
xgb_chars_encoder = TfidfVectorizer(max_features=20000, analyzer="char", ngram_range=(3,5), use_idf=True, sublinear_tf=True)
xgb_chars_encoder.fit(train_raw.texts)

# Prepare train & test set
X_train_xgb_chars = xgb_chars_encoder.transform(train_raw.texts)
X_test_xgb_chars = xgb_chars_encoder.transform(test_raw.texts)

import xgboost as xgb

train_dmat_chars = xgb.DMatrix(X_train_xgb_chars, pd.array(train_raw.labels).astype('category'))
test_dmat_chars = xgb.DMatrix(X_test_xgb_chars, pd.array(test_raw.labels).astype('category'))

params = {
    "booster": "gbtree",
    "device": "cpu",
    "objective": "binary:logistic",  # there is also binary:hinge but hinge does not output probability
    "tree_method": "hist",  # default to hist
    "device": "cuda",

    # Params for tree booster
    "eta": 0.3,
    "gamma": 0.0,  # Min loss achieved to split the tree
    "max_depth": 6,
    "reg_alpha": 0,
    "reg_lambda": 1,

}
evals_chars = [(train_dmat_chars, "train")]
iterations = 2000

model_xgb_chars = xgb.train(
    params = params,
    dtrain = train_dmat_chars,
    num_boost_round = iterations,
    evals = evals_chars,
    verbose_eval = 100
)

pred_xgb_chars_probs = model_xgb_chars.predict(test_dmat_chars)
result_xgb_chars = evaluate(test_raw.labels, pred_xgb_chars_probs > 0.5, pred_xgb_chars_probs)

XGBoostError: [20:00:12] /workspace/src/tree/updater_gpu_hist.cu:781: Exception in gpu_hist: [20:00:12] /workspace/src/c_api/../data/../common/device_helpers.cuh:431: Memory allocation error on worker 0: std::bad_alloc: cudaErrorMemoryAllocation: out of memory
- Free memory: 1327693824
- Requested memory: 2536918976

Stack trace:
  [bt] (0) /home/hapham/.pyenv/versions/3.11.5/envs/power/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x77f79a) [0x7f0c63ba179a]
  [bt] (1) /home/hapham/.pyenv/versions/3.11.5/envs/power/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x783994) [0x7f0c63ba5994]
  [bt] (2) /home/hapham/.pyenv/versions/3.11.5/envs/power/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0xb3737b) [0x7f0c63f5937b]
  [bt] (3) /home/hapham/.pyenv/versions/3.11.5/envs/power/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0xb3d674) [0x7f0c63f5f674]
  [bt] (4) /home/hapham/.pyenv/versions/3.11.5/envs/power/lib/python3.11/site-packages/xgboost/lib/libxg